In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

## View MobileNetV3 Architecture for Reference

In [2]:
weights = MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = mobilenet_v3_large(weights=weights)

print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

## Remove Last Layer

In [3]:
model_submodule_excl_last = list(model.children())[:-1]
model_submodule_excl_last = nn.Sequential(*model_submodule_excl_last)

model_submodule_excl_last

Sequential(
  (0): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

In [4]:
model_submodule_excl_last(torch.rand(10, 3, 224, 224)).shape

torch.Size([10, 960, 1, 1])

In [5]:
model_last_submodule = list(list(model.children())[-1].children())[:-1]  # Remove last layer from last submodule
model_last_submodule = nn.Sequential(*model_last_submodule)

model_last_submodule

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
)

In [6]:
# output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
# model_last_submodule(output)

In [7]:
output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
output = torch.flatten(output, 1)
model_last_submodule(output).shape

torch.Size([10, 1280])

## Import `SiameseDuplicateImageNetwork` for Testing

In [8]:
from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
model = SiameseDuplicateImageNetwork().to(device)

In [11]:
print(model)

SiameseDuplicateImageNetwork(
  (preprocess): ImageClassification(
      crop_size=[224]
      resize_size=[232]
      mean=[0.485, 0.456, 0.406]
      std=[0.229, 0.224, 0.225]
      interpolation=InterpolationMode.BILINEAR
  )
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): Hardswish()
          )
          (1): InvertedResidual(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
              )
              (1): Conv2dNo

In [12]:
train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [13]:
feature1, feature2, _ = next(iter(train_dataloader))
feature1, feature2

(tensor([[[[214, 214, 215,  ..., 223, 223, 223],
           [214, 214, 215,  ..., 223, 223, 223],
           [214, 214, 215,  ..., 223, 223, 223],
           ...,
           [194, 193, 193,  ..., 206, 201, 195],
           [193, 194, 195,  ..., 200, 201, 199],
           [195, 196, 197,  ..., 209, 202, 200]],
 
          [[207, 207, 208,  ..., 215, 215, 215],
           [207, 207, 208,  ..., 215, 215, 215],
           [207, 207, 208,  ..., 215, 215, 215],
           ...,
           [149, 148, 148,  ..., 130, 128, 122],
           [148, 149, 150,  ..., 124, 128, 126],
           [150, 151, 152,  ..., 133, 129, 127]],
 
          [[191, 191, 192,  ..., 204, 204, 204],
           [191, 191, 192,  ..., 204, 204, 204],
           [191, 191, 192,  ..., 204, 204, 204],
           ...,
           [107, 106, 106,  ...,  74,  73,  67],
           [106, 107, 108,  ...,  68,  73,  71],
           [108, 109, 110,  ...,  77,  74,  72]]],
 
 
         [[[184, 183, 180,  ..., 214, 214, 213],
         

In [14]:
feature1 = feature1.to(device)
feature2 = feature2.to(device)

output = model(feature1, feature2)
print(output)

C:\Users\ruiqinng\miniconda3\envs\dai\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor([[0.0972],
        [0.0985],
        [0.0963],
        [0.0994],
        [0.0978],
        [0.0968],
        [0.0979],
        [0.0973],
        [0.0961],
        [0.0976],
        [0.0972],
        [0.0968],
        [0.0974],
        [0.0990],
        [0.0951],
        [0.0980],
        [0.0968],
        [0.0960],
        [0.0967],
        [0.0980],
        [0.0967],
        [0.0975],
        [0.0961],
        [0.0950],
        [0.0963],
        [0.0982],
        [0.0978],
        [0.0964],
        [0.0981],
        [0.0958],
        [0.0965],
        [0.0971],
        [0.0953],
        [0.0981],
        [0.0970],
        [0.0975],
        [0.0956],
        [0.0983],
        [0.0981],
        [0.0969],
        [0.0961],
        [0.0955],
        [0.0974],
        [0.0966],
        [0.0977],
        [0.0977],
        [0.0967],
        [0.0964],
        [0.0956],
        [0.0983],
        [0.0973],
        [0.0961],
        [0.0957],
        [0.0971],
        [0.0972],
        [0

In [15]:
output.shape

torch.Size([64, 1])